# Using fully with Python

`ecodynelec` offers the possibility to being fully used via Python. This tutorial shows the different steps and possibilities, relying solely on operations doable within a python script or notebook. As the configuration of `ecodynelec` pipeline execution may benefit from more visual interaction, an alternative is suggested in the "[Using with a spreadsheet-based configuraion](https://ecodynelec.readthedocs.io/en/latest/examples/with_spreadsheet.html)" tutorial.

## Initialization

To download and install `ecodynelec` to being used as a python package, the user is referred to either the [getting started tutorial](https://ecodynelec.readthedocs.io/en/latest/examples/getting_started.html).

## Configuration

The configuration of `ecodynelec` is handled by the `parameter` module. 

In [1]:
from ecodynelec.parameter import Parameter # Import the class to manipulate parameters

Python is an object oriented language. Thus a specific configuration can be built and stored in a `Parameter` object that will be called `my_config` in this tutorial. The next cell only initializes the configuration object with the default parameters.

In [2]:
my_config = Parameter() # Initialize a configuration object

Before modifying the configuration, let's have a look at this default setting

In [ ]:
print(my_config)

The configuration is composed of 3 parts. The detail about the meaning of each is developed in the [input data section](https://ecodynelec.readthedocs.io/en/latest/data_input/parameters.html). Essentially:
* the <u>first block</u> contains the elements to configure the execution itself. These elements are directly available and modifiable with the syntax `my_config.element`.
* the <u>second block</u> deals with all paths to information files, directory containing information, or where to write and save information before, during and after the computation. It is accessible with the syntax `my_config.path.element`
* the <u>third block</u> deals with information related to the ENTSO-E server, as electricity data from the ENTSO-E server is at the center of `ecodynelec`. More on this topic is covered on the next paragraph and on the dedicated [downloading tutorial](https://ecodynelec.readthedocs.io/en/latest/examples/downloading.html).

The next cell partly modifies the execution configuration.
* First the starting date is modified. Note that objects of the `Parameter` class will verify if this element is a date, and will raise an error if the format is not recognized.
* Then we modify the size of time step (frequency) for the computation. Possibilities are specified in the [input data section](https://ecodynelec.readthedocs.io/en/latest/data_input/parameters.html).
* Third in this example, the auto-completing feature is turned off.

In [3]:
## Change the starting date
my_config.start = '2017-02-01 00:00'

## Change the time step
my_config.freq = "15min"

## Change the coutry list
my_config.ctry = ['AT','CH','DE','FR','IT', 'CZ']

## Turn off the auto-complete
my_config.data_cleaning = False

The next cell partly modifies file path configuration. Here we modify the location of directories containing data downloaded from the ENTSO-E database.

In [4]:
# Indicate where to find generation data
my_config.path.generation = "./test_data/downloads/generations/"

# Indicate where to find exchange data
my_config.path.exchanges = "./test_data/downloads/exchanges/"

Note that, for the `generation`, `exchanges` and `savedir` paths, the specified directory *will be created if it does not already exist*. For every other file path element, *a default file* is used if nothing is specified, and an error is returned if the information passed does not correspond to any existing file on your local machine.

### Downloading Entso-E data

The [downloading tutorial](https://ecodynelec.readthedocs.io/en/latest/examples/downloading.html) covers the specificities about how to download the ENTSO-E data or include the download as part of the `ecodynelec` pipeline execution. This feature is not triggered per default and `ecodynelec` is expecting to find already downloaded ENTSO-E files.

## Execution

`ecodynelec` is build out of a myriad of modules that can be used relatively independently, under the condition that inputs data is shaped the correct way. Fortunately, the entire pipeline starting from a set of parameters and computing down to the calculation of impact metrics.

The usage of this entire pipeline is demonstrated below. This pipeline allows to save results into files (c.f. paragraph on [configuration](https://ecodynelec.readthedocs.io/en/latest/examples/with_python.html#configuration)). However results are also always returned for further in-script use. These results are stored in the `impacts` variable for later paragraphs in this tutorial.

In [ ]:
from ecodynelec.pipelines import execute
from ecodynelec.progress_info import ProgressInfo

In [ ]:
impacts = execute(config=my_config, progress_bar=ProgressInfo())

## Outcome and Visualization

The outcome is stored in files and returned for further in-script use. In the previous section, results were stored in the `impacts` variable. The current section highlights the content returned and shows some basic possibilities for data visualization.

In [ ]:
import numpy as np
import pandas as pd

### Description of the outcome

The `impacts` variable contains a collection of tables. This collection is a `dict` with one `Global` key, and one other key per impact category:

In [ ]:
print(impacts.keys())

The `Global` table is the *sum across all technologies* for each index, as it is shown for the first few time steps:

In [ ]:
display(impacts['Global'].head())

The other tables are, for each impact category, the breakdown into all possible sources:

In [ ]:
for i in impacts: # Iterate for all impact categories
    if i=='Global': continue; # Skip the Global, already visualized above.
    
    print(f"#############\nimpacts for {i}:")
    display( impacts[i].head(3).T ) # Transpose table for readability

### Group per country

The following piece of code suggests a basic visualization of the Climate Change category, grouping the results per country of origin of the tracked electricity.

In [ ]:
impacts['Climate Change'].columns

In [ ]:
# Filters the CH
cli = impacts['Climate Change']
lcs = cli.loc[:,[not src.endswith('_CH') for src in cli.columns]]
cli2 = cli.drop(lcs, axis=1)

In [ ]:
cli2.keys()

In [ ]:
def compute_per_country(results):
    """Function to group results per country"""
    countries = np.unique([c.split("_")[-1] for c in results.columns]) # List of countries
    
    per_country = []
    for c in countries:
        cols = [k for k in results.columns if k[-3:]==f"_{c}"]
        per_country.append(pd.Series(results.loc[:,cols].sum(axis=1), name=c))
        
    return pd.concat(per_country,axis=1)

In [ ]:
gwp_per_country = compute_per_country(impacts['Climate Change']) # Group Climate Change index impacts per country
gwp_per_country.plot.area(figsize=(12,4), legend='reverse', color=['r','w','y','b','c','k'],
                          title="Some visualization of the GWP aggregated per country"); # Build the graph

In [ ]:
gwp_per_country['CH'].plot()

### Group per production type

The following piece of code suggests a basic visualization of the Climate Change category, grouping the results per technology of origin of the tracked electricity.

In [ ]:
impacts.keys()

In [ ]:
def compute_per_type(results):
    """Function to group datasets per type of unit, regardless of the country of origin"""
    unit_list = np.unique([k[:-3] if k[-3]=="_" else k for k in results.columns]) # List the different production units
    
    per_unit = []
    for u in unit_list:
        cols = [k for k in results.columns if k[:-3]==u] # collect the useful columns
        per_unit.append(pd.Series(results.loc[:,cols].sum(axis=1), name=u)) # aggregate

    return pd.concat(per_unit,axis=1)

In [ ]:
es13_per_type = compute_per_type(impacts['Climate Change']) # Group Climate Change index impacts per country
es13_per_type.plot.area(figsize=(12,8), legend='reverse',
                        title="Some visualization of the Climate Change index aggregated per source"); # Build the graph